# Getting Started with MCPost

This tutorial provides a gentle introduction to MCPost, covering both Global Sensitivity Analysis (GSA) and Monte Carlo integration capabilities.

## What is MCPost?

MCPost is a Python package for post-analysis of Monte Carlo samples, providing:

- **Global Sensitivity Analysis (GSA)**: Identify which input parameters most influence your model outputs
- **Monte Carlo Integration**: Compute integrals using various Monte Carlo and quasi-Monte Carlo methods
- **Modern Python Package**: Type hints, comprehensive documentation, and extensive testing

## Installation

```bash
# Basic installation
pip install mcpost

# With visualization support
pip install mcpost[viz]
```

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from mcpost import gsa_pipeline, monte_carlo_integral, qmc_integral_auto

# Set random seed for reproducibility
np.random.seed(42)

print("MCPost successfully imported!")

## Part 1: Global Sensitivity Analysis

Let's start with a simple example using a polynomial function to understand how GSA works.

In [ ]:
# Define a simple test function
def polynomial_function(X):
    """
    Simple polynomial: f(x1, x2, x3) = x1^2 + 2*x2 + 0.1*x3
    
    We expect x2 to be most influential, x1 moderately influential, 
    and x3 to have minimal influence.
    """
    x1, x2, x3 = X[:, 0], X[:, 1], X[:, 2]
    return x1**2 + 2*x2 + 0.1*x3

# Generate parameter samples
n_samples = 1000
X = np.random.uniform(-1, 1, (n_samples, 3))  # 3 parameters in [-1, 1]

# Evaluate function
y = polynomial_function(X)
Y = y.reshape(-1, 1)  # GSA expects 2D array

print(f"Generated {n_samples} samples")
print(f"Parameter ranges: {X.min(axis=0)} to {X.max(axis=0)}")
print(f"Output range: [{y.min():.3f}, {y.max():.3f}]")

In [ ]:
# Run GSA analysis
param_names = ["x1", "x2", "x3"]
feature_names = ["polynomial"]

print("Running GSA analysis...")
results = gsa_pipeline(
    X, Y,
    param_names=param_names,
    feature_names=feature_names,
    scaler="minmax",
    enable_sobol=True,
    enable_gp=True,
    enable_perm=True,
    make_pdp=False,  # Skip PDPs for this simple example
    N_sobol=2048
)

# Display results
sensitivity_table = results["results"]["polynomial"]["table"]
print("\nSensitivity Analysis Results:")
print(sensitivity_table)

In [ ]:
# Visualize the results
from mcpost import plot_sensitivity_metrics

fig, ax = plot_sensitivity_metrics(
    sensitivity_table,
    title="Polynomial Function - Sensitivity Analysis"
)
plt.show()

print("\nInterpretation:")
print("- x2 shows highest sensitivity (expected - linear coefficient of 2)")
print("- x1 shows moderate sensitivity (quadratic term)")
print("- x3 shows low sensitivity (small coefficient of 0.1)")

## Part 2: Understanding GSA Metrics

MCPost provides several sensitivity metrics. Let's understand what each one tells us:

In [ ]:
# Extract individual metrics for interpretation
table = sensitivity_table

print("GSA Metrics Explanation:")
print("=" * 25)

for param in param_names:
    print(f"\nParameter: {param}")
    print(f"  MI (Mutual Information): {table.loc[param, 'MI']:.3f}")
    print(f"    → Measures statistical dependence (higher = more influential)")
    
    print(f"  dCor (Distance Correlation): {table.loc[param, 'dCor']:.3f}")
    print(f"    → Captures nonlinear relationships (0 = independent, 1 = perfectly dependent)")
    
    if 'PermMean' in table.columns:
        print(f"  Perm (Permutation Importance): {table.loc[param, 'PermMean']:.3f}")
        print(f"    → Model-agnostic importance (decrease in performance when shuffled)")
    
    if 'Si' in table.columns:
        print(f"  Si (First-order Sobol): {table.loc[param, 'Si']:.3f}")
        print(f"    → Main effect contribution to output variance")
        
        print(f"  STi (Total Sobol): {table.loc[param, 'STi']:.3f}")
        print(f"    → Total effect including interactions")
    
    if '1/ARD_LS' in table.columns:
        print(f"  1/ARD_LS (GP Relevance): {table.loc[param, '1/ARD_LS']:.3f}")
        print(f"    → Gaussian Process automatic relevance determination")

## Part 3: Monte Carlo Integration

Now let's explore MCPost's integration capabilities with a simple example.

In [ ]:
# Define integration problem: E[x^2] where x ~ N(0,1)
# Analytical solution: 1.0

def integrand(theta):
    """Function to integrate: f(x) = x^2"""
    return theta[:, 0]**2

def target_pdf(theta):
    """Standard normal PDF"""
    return np.exp(-0.5 * theta[:, 0]**2) / np.sqrt(2 * np.pi)

print("Integration Problem: E[X^2] where X ~ N(0,1)")
print("Analytical solution: 1.0")
print()

In [ ]:
# Method 1: Standard Monte Carlo
n_samples = 5000
theta_samples = np.random.normal(0, 1, (n_samples, 1))
f_values = integrand(theta_samples)

mc_result = monte_carlo_integral(theta_samples, f_values, target_pdf)

print("Standard Monte Carlo:")
print(f"  Integral estimate: {mc_result['integral'][0]:.6f}")
print(f"  Uncertainty: {mc_result['uncertainty'][0]:.6f}")
print(f"  Effective sample size: {mc_result['effective_sample_size']:.0f}")
print(f"  Error: {abs(mc_result['integral'][0] - 1.0):.6f}")

In [ ]:
# Method 2: Quasi-Monte Carlo (automatic)
qmc_result = qmc_integral_auto(
    N_samples=n_samples,
    N_params=1,
    data_func=integrand,
    p_target=target_pdf,
    bounds=[(-4, 4)],  # Integration bounds
    method='sobol'
)

print("\nQuasi-Monte Carlo (Sobol):")
print(f"  Integral estimate: {qmc_result['integral'][0]:.6f}")
print(f"  Effective sample size: {qmc_result['effective_sample_size']:.0f}")
print(f"  Error: {abs(qmc_result['integral'][0] - 1.0):.6f}")

In [ ]:
# Compare convergence rates
sample_sizes = [100, 500, 1000, 2000, 5000]
mc_errors = []
qmc_errors = []

print("\nConvergence Comparison:")
print(f"{'N_samples':<10} {'MC Error':<12} {'QMC Error':<12}")
print("-" * 35)

for n in sample_sizes:
    # Monte Carlo
    theta = np.random.normal(0, 1, (n, 1))
    f_vals = integrand(theta)
    mc_res = monte_carlo_integral(theta, f_vals, target_pdf)
    mc_error = abs(mc_res['integral'][0] - 1.0)
    mc_errors.append(mc_error)
    
    # QMC
    qmc_res = qmc_integral_auto(
        N_samples=n, N_params=1, data_func=integrand,
        p_target=target_pdf, bounds=[(-4, 4)]
    )
    qmc_error = abs(qmc_res['integral'][0] - 1.0)
    qmc_errors.append(qmc_error)
    
    print(f"{n:<10} {mc_error:<12.6f} {qmc_error:<12.6f}")

print("\nNote: QMC often shows better convergence for smooth integrands")

## Part 4: Putting It All Together

Let's combine GSA and integration in a practical example: analyzing a model's sensitivity and computing expected values.

In [ ]:
# Define a more complex model
def engineering_model(X):
    """
    Simplified engineering model: stress in a beam
    
    Parameters:
    - x1: Load [kN]
    - x2: Length [m] 
    - x3: Width [cm]
    - x4: Height [cm]
    
    Output: Maximum stress [MPa]
    """
    load = X[:, 0]      # Load in kN
    length = X[:, 1]    # Length in m
    width = X[:, 2]     # Width in cm
    height = X[:, 3]    # Height in cm
    
    # Moment of inertia (simplified)
    I = width * height**3 / 12
    
    # Maximum bending moment
    M_max = load * length**2 / 8  # Simply supported beam, uniform load
    
    # Maximum stress (bending)
    stress = M_max * (height/2) / I * 1000  # Convert to MPa
    
    return stress

# Parameter ranges (realistic engineering values)
param_ranges = {
    'Load': [10, 50],      # kN
    'Length': [2, 8],      # m
    'Width': [10, 30],     # cm
    'Height': [15, 40],    # cm
}

print("Engineering Model: Beam Stress Analysis")
print("Parameter ranges:")
for param, (low, high) in param_ranges.items():
    print(f"  {param}: [{low}, {high}]")

In [ ]:
# Generate samples and run GSA
n_samples = 2000
X_eng = np.zeros((n_samples, 4))

# Sample parameters uniformly
for i, (param, (low, high)) in enumerate(param_ranges.items()):
    X_eng[:, i] = np.random.uniform(low, high, n_samples)

# Evaluate model
stress = engineering_model(X_eng)
Y_eng = stress.reshape(-1, 1)

print(f"Stress range: [{stress.min():.1f}, {stress.max():.1f}] MPa")

# Run GSA
param_names_eng = list(param_ranges.keys())
results_eng = gsa_pipeline(
    X_eng, Y_eng,
    param_names=param_names_eng,
    feature_names=["Stress_MPa"],
    scaler="minmax",
    enable_sobol=True,
    make_pdp=False
)

print("\nBeam Stress Sensitivity Analysis:")
print(results_eng["results"]["Stress_MPa"]["table"])

In [ ]:
# Compute expected stress using integration
def uniform_pdf(theta):
    """Uniform PDF over parameter ranges"""
    # Volume of parameter space
    volume = 1.0
    for low, high in param_ranges.values():
        volume *= (high - low)
    return np.ones(len(theta)) / volume

def stress_integrand(theta):
    """Stress function for integration"""
    return engineering_model(theta).reshape(-1, 1)

# Compute expected stress
bounds = list(param_ranges.values())
expected_stress = qmc_integral_auto(
    N_samples=4096,
    N_params=4,
    data_func=stress_integrand,
    p_target=uniform_pdf,
    bounds=bounds
)

print(f"\nExpected stress: {expected_stress['integral'][0]:.2f} MPa")
print(f"Sample mean stress: {stress.mean():.2f} MPa")
print(f"Difference: {abs(expected_stress['integral'][0] - stress.mean()):.2f} MPa")

## Summary

In this tutorial, you learned:

1. **GSA Basics**: How to run sensitivity analysis and interpret different metrics
2. **Integration Methods**: Standard MC vs. QMC and their convergence properties
3. **Practical Application**: Combining GSA and integration for engineering analysis

### Key Takeaways:

- **GSA helps identify** which parameters most influence your model outputs
- **Multiple metrics** (MI, dCor, Sobol, etc.) provide different perspectives on sensitivity
- **QMC methods** often converge faster than standard Monte Carlo for smooth functions
- **MCPost provides** a unified interface for both GSA and integration tasks

### Next Steps:

- Explore the `examples/` directory for more complex applications
- Check out the API documentation for advanced features
- Try MCPost on your own models and datasets!

### Resources:

- [MCPost Documentation](https://mcpost.readthedocs.io/)
- [Examples Gallery](../examples/)
- [API Reference](https://mcpost.readthedocs.io/en/latest/api/)